# Full supervised training

To establish a goal line the Inception network is trained in a full supervised fashion.


In [ ]:
from deepal_for_ecg.data.load import PTBXLDataLoader
from deepal_for_ecg.data.active_learning import PTBXLActiveLearningDataModule
from deepal_for_ecg.models.inception_network import InceptionNetworkBuilder, InceptionNetworkConfig
from deepal_for_ecg.train.ptbxl import PTBXLTrainer
from deepal_for_ecg.util import improve_gpu_capacity

In [ ]:
improve_gpu_capacity()

## Data

Load the data and prepare the data module.

In [ ]:
data_loader = PTBXLDataLoader(load_saved_data=True, saved_data_base_dir="../data/saved/")
data_loader.load_data()

In [ ]:
data_module = PTBXLActiveLearningDataModule(
    train_samples=data_loader.X_train, 
    test_samples=data_loader.X_test, 
    val_samples=data_loader.X_valid, 
    train_labels_12sl=data_loader.Y_train_12sl, 
    train_labels_ptb_xl=data_loader.Y_train_ptb_xl,
    test_labels=data_loader.Y_test,
    val_labels=data_loader.Y_valid
)

In [ ]:
data_module_state = data_module.state_dict()
data_module.update_annotations(buy_idx_ptb_xl=data_module_state["unlabeled_indices"], buy_idx_12sl=set())

## Inception network

### Build the models

Build the Inception networks.

In [ ]:
builder = InceptionNetworkBuilder()
config = InceptionNetworkConfig()

ensemble = [builder.build_model(config) for _ in range(5)]

### Training
Train all five models of the Inception network.

In [ ]:
num_classes = 93
trainer = PTBXLTrainer(ensemble[0], model_name="full_supervised_inception_1", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

In [ ]:
trainer = PTBXLTrainer(ensemble[1], model_name="full_supervised_inception_2", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

In [ ]:
trainer = PTBXLTrainer(ensemble[2], model_name="full_supervised_inception_3", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

In [ ]:
trainer = PTBXLTrainer(ensemble[3], model_name="full_supervised_inception_4", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

In [ ]:
trainer = PTBXLTrainer(ensemble[4], model_name="full_supervised_inception_5", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
trainer.fit(data_module.train_dataset, data_module.validation_dataset)

## Testing

Test the Inception network.

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
loss_object = keras.losses.BinaryCrossentropy()
test_loss = keras.metrics.Mean(name="test_loss")
test_accuracy = keras.metrics.BinaryAccuracy(name="test_accuracy")
test_auc = keras.metrics.AUC(multi_label=True, name="test_auc", num_labels=num_classes)

@tf.function
def test_step_ensemble(time_series_batch, label_batch, models_to_test):
    ensemble_predictions = []
    for model_to_test in models_to_test:
        sliding_window_predictions = []
        for sliding_window in time_series_batch:
            sliding_window_predictions.append(model_to_test(sliding_window, training=False))
    
        ensemble_predictions.append(tf.reduce_max(sliding_window_predictions, axis=0))
    predictions = tf.reduce_mean(ensemble_predictions, axis=0)
    t_loss = loss_object(label_batch, predictions)

    test_loss(t_loss)
    test_accuracy(label_batch, predictions)
    test_auc(label_batch, predictions)

In [ ]:
model_names = [f"full_supervised_inception_{x}" for x in range(1, 6)]

models = [keras.models.load_model(f"../models/{model_name}/best_model.keras") for model_name in model_names]

test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for test_series, test_labels in data_module.test_dataset.batch(128):
    test_step_ensemble(test_series, test_labels, models)

print(
    f'Test data stats for model ensemble: '
    f'Loss: {test_loss.result()}, '
    f'Accuracy: {test_accuracy.result() * 100}, '
    f'AUC: {test_auc.result() * 100}'
)

## H-Inception network
Compare it to the H-Inception network, which has some handcrafted filters.

### Build the models

In [ ]:
h_builder = InceptionNetworkBuilder()
config = InceptionNetworkConfig(use_handcrafted_filters=True)
h_ensemble = [builder.build_model(config) for _ in range(5)]

### Train all H-Inception models

In [ ]:
for i, model in enumerate(h_ensemble):
    trainer = PTBXLTrainer(model, model_name=f"full_supervised_h_inception_{i+1}", num_labels=num_classes, model_base_dir="../models", log_base_dir="../logs")
    trainer.fit(data_module.train_dataset, data_module.validation_dataset)

### Test models

In [ ]:
model_names = [f"full_supervised_h_inception_{x}" for x in range(1, 6)]

models = [keras.models.load_model(f"../models/{model_name}/best_model.keras") for model_name in model_names]

test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for test_series, test_labels in data_module.test_dataset.batch(128):
    test_step_ensemble(test_series, test_labels, models)

print(
    f'Test data stats for model ensemble: '
    f'Loss: {test_loss.result()}, '
    f'Accuracy: {test_accuracy.result() * 100}, '
    f'AUC: {test_auc.result() * 100}'
)